### Get access token
[Documentation](https://documentation.dataspace.copernicus.eu/APIs/Token.html)

### OAuth Setup
[Docs](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#python), [Credentials](https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings)


In [149]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import json
import pathlib
from osgeo import gdal
from copernicus import Client, get_labels, get_sentinel2_product_ids, download_sentinel2_products
import building_dection as bd



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
credentials = json.load(open('credentials.json'))
client = Client(credentials['username'], credentials['password'])

### Define Project Paths 

In [151]:
sn7_path = pathlib.Path('D:\snap_data\sn7')
data_path = sn7_path / 'data'
train_path = sn7_path / 'SN7_buildings_train' / 'train'
sentinel_path = sn7_path / 'sentinel-2'
vector_label_dir = sn7_path / 'vector_labels'

### Get Building Label Files

In [152]:
labels = get_labels(train_path)[:20]

# copy labels files to vector_label_dir
for label in labels:
    shutil.copy(label, vector_label_dir / label.name)

labels = [ vector_label_dir / label.name for label in labels]



### Init data dir and location dirs

In [153]:
label = labels[2]
loc_name  = str(label).split("_mosaic_")[-1].split("_Buildings")[0].split("-")[1]
label_name = str(label).split("_mosaic_")[-1].split("_Buildings")[0]
loc_path = data_path / loc_name
print(loc_name, label_name, loc_path)
label

0358E L15-0358E-1220N_1433_3310_13 D:\snap_data\sn7\data\0358E


WindowsPath('D:/snap_data/sn7/vector_labels/global_monthly_2020_01_mosaic_L15-0358E-1220N_1433_3310_13_Buildings.geojson')

In [154]:
sn7_locations = []
for label in labels:
    label_name = str(label).split("_mosaic_")[-1].split("_Buildings")[0]
    loc_path = data_path / str(label).split("_mosaic_")[-1].split("_Buildings")[0].split("-")[1]
    sn7_loc = bd.SN7_Location(loc_path, label_name)
    sn7_loc.init_labels(label)
    sn7_locations.append(sn7_loc)

In [155]:
sn7_locations[0].l1c_path

WindowsPath('None')

### Get S2-L1C products

In [156]:
for sn7_loc in sn7_locations:
    try:
        sn7_loc.init_l1c(client, sentinel_path)
    except:
        print("No L1C Product Found for: ", sn7_loc.label)
        # remove the location from the list
        sn7_locations.remove(sn7_loc)

Product S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE already downloaded
Product S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE already downloaded
No L1C Product Found for:  L15-0357E-1223N_1429_3296_13
Product S2B_MSIL1C_20200117T184709_N0208_R070_T11TNJ_20200117T203714.SAFE already downloaded
No L1C Product Found for:  L15-0361E-1300N_1446_2989_13
Product S2A_MSIL1C_20200111T173701_N0500_R055_T13SFR_20230428T081153.SAFE already downloaded
No L1C Product Found for:  L15-0434E-1218N_1736_3318_13
Product S2A_MSIL1C_20200121T160511_N0500_R054_T17RNL_20230426T201445.SAFE already downloaded
Product S2A_MSIL1C_20200105T154541_N0500_R111_T17PPL_20230421T075135.SAFE already downloaded
Product S2A_MSIL1C_20200115T154541_N0208_R111_T18QTE_20200115T190541.SAFE already downloaded
No L1C Product Found for:  L15-0586E-1127N_2345_3680_13
No L1C Product Found for:  L15-0614E-0946N_2459_4406_13
Product S2B_MSIL1C_20200103T141729_N0500_R010_T20MRB_20230618T162034.

In [157]:
sn7_locations[0].l1c_path

WindowsPath('D:/snap_data/sn7/data/0331E/L1C.SAFE')

In [25]:
locs = {}
for k,v in downloaded_products.items():
    locs[k] = v['SAFE_path']
    break
    
modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    # {
    #     "flag": "SEN2COR_MODIFY_LUT_SEASON",
    #     "value": "WINTER",
    #     "info": "Look Up Table Mid Latitude Season set to WINTER",
    #     "name": "LUT_SEASON_WINTER",
    # },
]


In [26]:
import sys
sys.path.append("../")
from l2a_analysis import L2A_Analysis

l2a_analysis = L2A_Analysis(report_name="sn7_cop", base_input_dir=sentinel_path, base_output_dir=sn7_path/"reports", resolution=10)
l2a_analysis.set_locations(locs)
l2a_analysis.set_modifications(modifications)


In [27]:
for loc, product in downloaded_products.items():
    roi = bd.get_region_of_interest(product["raster_label_path"], product["SAFE_path"])

    
    sen2cor_roi = bd.transform_roi_for_sen2cor(roi)
    l2a_analysis.add_region_of_interest(loc, sen2cor_roi)
    break

label_extent: ulx: 615497.0	uly: 4206249.0	lrx: 619397.0	lry: 4202389.0
SAFE_extent: ulx:  600000	uly: 4300020	lrx: 709800	lry: 4190220


In [28]:
l2a_analysis.run_sen2cor()


Running sen2cor for L15-0331E-1257N_1327_3160_13: Reference
Running command:  C:\Users\tlaus\Documents\local\semester_thesis\Sen2Cor-02.11.00-win64\L2A_Process.bat D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13\S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE --resolution 10 --GIP_L2A C:\Users\tlaus\Documents\sen2cor\2.11\cfg\L2A_GIPP_custom.xml --output_dir D:\snap_data\sn7\reports/space_net_7_10_2020_copernicus/L15-0331E-1257N_1327_3160_13/reference
Running sen2cor for L15-0331E-1257N_1327_3160_13: LUT_AEROSOL_MARITIME
Running command:  C:\Users\tlaus\Documents\local\semester_thesis\Sen2Cor-02.11.00-win64\L2A_Process.bat D:\snap_data\sn7\sentinel-2\L15-0331E-1257N_1327_3160_13\S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE --resolution 10 --GIP_L2A C:\Users\tlaus\Documents\sen2cor\2.11\cfg\L2A_GIPP_custom.xml --output_dir D:\snap_data\sn7\reports/space_net_7_10_2020_copernicus/L15-0331E-1257N_1327_3160_13/LUT_AEROSOL_MARITIME
